Lambda School Data Science

*Unit 4, Sprint 1, Module 3*

---

# Document Classification (Assignment)

This notebook is for you to practice skills during lecture.

Today's guided module project and assignment will be different. You already know how to do classification. You ready know how to extract features from documents. So? That means you're ready to combine and practice those skills in a kaggle competition. We we will open with a five minute sprint explaining the competition, and then give you 25 minutes to work. After those twenty five minutes are up, I will give a 5-minute demo an NLP technique that will help you with document classification (*and **maybe** the competition*).

Today's all about having fun and practicing your skills.

## Sections
* <a href="#p1">Part 1</a>: Text Feature Extraction & Classification Pipelines
* <a href="#p2">Part 2</a>: Latent Semantic Indexing
* <a href="#p3">Part 3</a>: Word Embeddings with Spacy
* <a href="#p4">Part 4</a>: Post Lecture Assignment

# Text Feature Extraction & Classification Pipelines (Learn)
<a id="p1"></a>

## Follow Along 

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model (try using the pipe method I just demoed)

### Load Competition Data

In [1]:
# Import Statements
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_20newsgroups
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
# You may need to change the path
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')

In [2]:
print(test.shape)
test.head()

(1022, 2)


,id,description
0,3461,\nStyle: Speyside single malt scotch Color: Wa...
1,2604,"\nVery bright and lively, with a nice balance ..."
2,3341,\nA new oloroso-forward Chivas positioned to s...
3,3764,\nAged in bourbon casks and then enhanced in R...
4,2306,\nThere is a freshness to the wood on the nose...


In [3]:
print(train.shape)
train.head()

(4087, 3)


,id,description,ratingCategory
0,1321,"\nSometimes, when whisky is batched, a few lef...",1
1,3861,\nAn uncommon exclusive bottling of a 6 year o...,0
2,655,\nThis release is a port version of Amrut’s In...,1
3,555,\nThis 41 year old single cask was aged in a s...,1
4,1965,"\nQuite herbal on the nose, with aromas of dri...",1


In [4]:
x_train = train["description"]
y_train = train["ratingCategory"]

### Define Pipeline Components

In [7]:
# Create Pipeline Components

vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
rfc = RandomForestClassifier()

# Define the Pipeline
pipe = Pipeline([
                 #Vectorizer
                 ('vect', vect),
                 # Classifier
                 ('clf', rfc)
                ])
# The pipeline puts together a bunch fit then transform,fit then predict. 

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [7]:
parameters = {
    'vect__max_df': (0.5,0.75, 1.0),
    'vect__min_df': (.05, .08, 1),
    'vect__max_features': (500, 1000, 2000),
    'clf__n_estimators':(20,50,100),
    'clf__max_depth':(10,15,20,25)
}

grid_search = GridSearchCV(pipe,parameters, cv=2, n_jobs=-1, verbose=1)
grid_search.fit(x_train, y_train)

Fitting 2 folds for each of 324 candidates, totalling 648 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  26 tasks      | elapsed:    6.2s
[Parallel(n_jobs=-1)]: Done 176 tasks      | elapsed:   25.0s
[Parallel(n_jobs=-1)]: Done 426 tasks      | elapsed:   57.8s
[Parallel(n_jobs=-1)]: Done 648 out of 648 | elapsed:  1.5min finished


GridSearchCV(cv=2, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('vect',
                                        TfidfVectorizer(analyzer='word',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 2),
                                                        no

In [8]:
grid_search.best_score_

0.7286525848848582

### Make a Submission File
*Note:* In a typical Kaggle competition, you are only allowed two submissions a day, so you only submit if you feel you cannot achieve higher test accuracy. For this competition the max daily submissions are capped at **20**. Submit for each demo and for your assignment. 

In [6]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

NameError: name 'grid_search' is not defined

In [10]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [11]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [12]:
subNumber = 0

In [13]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

You're trying to achieve a minimum of 70% Accuracy on your model.

In [8]:
print(grid_search.best_score_)
grid_search.best_params_

NameError: name 'grid_search' is not defined

## Latent Semantic Indexing (Learn)
<a id="p2"></a>

## Follow Along
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
4. Make a submission to Kaggle 


### Define Pipeline Components

In [9]:
# Import

from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=50, 
                   algorithm='randomized',
                   n_iter=40)

In [10]:
vect = TfidfVectorizer(stop_words='english', ngram_range=(1,2))
lsi = Pipeline([('vect', vect), ('svd', svd)])
rfc = RandomForestClassifier()

pipe = Pipeline([('lsi', lsi), ('clf', rfc)])

### Define Your Search Space
You're looking for both the best hyperparameters of your vectorizer and your classification model. 

In [14]:
parameters = { 
    'lsi__svd__n_components': [75,100,125],
    'lsi__vect__max_df':[1,2,3,4,5],
    'lsi__vect__min_df': [1,2,3],
    'lsi__vect__max_features': [1000, 5000],
    'clf__n_estimators':[100,1000],
    'clf__max_depth':[15,100]
}

grid_search = GridSearchCV(pipe,parameters, cv=3, n_jobs=-1, verbose=3)
grid_search.fit(x_train, y_train)

Fitting 3 folds for each of 360 candidates, totalling 1080 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done   8 tasks      | elapsed:    4.0s
[Parallel(n_jobs=-1)]: Done 104 tasks      | elapsed:   33.3s
[Parallel(n_jobs=-1)]: Done 264 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 488 tasks      | elapsed:  7.2min
[Parallel(n_jobs=-1)]: Done 776 tasks      | elapsed: 10.6min
[Parallel(n_jobs=-1)]: Done 1080 out of 1080 | elapsed: 19.1min finished


GridSearchCV(cv=3, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('lsi',
                                        Pipeline(memory=None,
                                                 steps=[('vect',
                                                         TfidfVectorizer(analyzer='word',
                                                                         binary=False,
                                                                         decode_error='strict',
                                                                         dtype=<class 'numpy.float64'>,
                                                                         encoding='utf-8',
                                                                         input='content',
                                                                         lowercase=True,
                                                                         max_df=1.0,
             

### Make a Submission File

In [15]:
# Predictions on test sample
pred = grid_search.predict(test['description'])

In [16]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [18]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [17]:
subNumber = 9

In [19]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model
submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

Continue to apply Latent Semantic Indexing (LSI) to various datasets. 

In [21]:
print(grid_search.best_score_)
grid_search.best_params_

0.7080992340396802


{'clf__max_depth': 15,
 'clf__n_estimators': 1000,
 'lsi__svd__n_components': 75,
 'lsi__vect__max_df': 5,
 'lsi__vect__max_features': 5000,
 'lsi__vect__min_df': 2}

# Word Embeddings with Spacy (Learn)
<a id="p3"></a>

## Follow Along

In [27]:
# Apply to your Dataset

from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import GradientBoostingClassifier
from scipy.stats import randint

param_dist = {
    
    'max_depth' : randint(3,10),
    'min_samples_leaf': randint(2,15)
}

In [26]:
# Continue Word Embedding Work Here
import spacy
nlp = spacy.load("en_core_web_lg")

In [28]:
def get_word_vectors(docs):
    return [nlp(doc).vector for doc in docs]

In [29]:
X = get_word_vectors(train['description'])
len(X) == len(train['description'])

True

In [30]:
X_test = get_word_vectors(test['description'])
rfc.fit(X, train['ratingCategory'])

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [31]:
rfc.score(X, train['ratingCategory'])

1.0

In [32]:
rfc.predict(X_test)

array([1, 1, 1, ..., 1, 1, 0], dtype=int64)

In [33]:
test['ratingCategory'] = rfc.predict(X_test)

In [34]:
test[['id', 'ratingCategory']].to_csv(f'./data/submission10.csv', header=True, index=False)

### Make a Submission File

In [22]:
# Predictions on test sample
pred = rfc.predict(test['id'])

NotFittedError: This RandomForestClassifier instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [23]:
submission = pd.DataFrame({'id': test['id'], 'ratingCategory':pred})
submission['ratingCategory'] = submission['ratingCategory'].astype('int64')

In [24]:
# Make Sure the Category is an Integer
submission.head()

,id,ratingCategory
0,3461,1
1,2604,1
2,3341,1
3,3764,1
4,2306,1


In [41]:
# Save your Submission File
# Best to Use an Integer or Timestamp for different versions of your model

submission.to_csv(f'./data/submission{subNumber}.csv', index=False)
subNumber += 1

## Challenge

What you should be doing now:
1. Join the Kaggle Competition
2. Download the data
3. Train a model & try: 
    - Creating a Text Extraction & Classification Pipeline
    - Tune the pipeline with a `GridSearchCV` or `RandomizedSearchCV`
    - Add some Latent Semantic Indexing (lsi) into your pipeline. *Note:* You can grid search a nested pipeline, but you have to use double underscores ie `lsi__svd__n_components`
    - Try to extract word embeddings with Spacy and use those embeddings as your features for a classification model.
4. Make a submission to Kaggle 

# Post Lecture Assignment
<a id="p4"></a>

Your primary assignment this afternoon is to achieve a minimum of 70% accuracy on the Kaggle competition. Once you have achieved 70% accuracy, please work on the following: 

1. Research "Sentiment Analysis". Provide answers in markdown to the following questions: 
    - What is "Sentiment Analysis"? 
    - Is Document Classification different than "Sentiment Analysis"? Provide evidence for your response
    - How do create labeled sentiment data? Are those labels really sentiment?
    - What are common applications of sentiment analysis?
2. Research our why word embeddings worked better for the lecture notebook than on the whiskey competition.
    - This [text classification documentation](https://developers.google.com/machine-learning/guides/text-classification/step-2-5) from Google might be of interest
    - Neural Networks are becoming more popular for document classification. Why is that the case?

- 1) Sentiment Analysis research

Sentiment Analysis is the process of analyzing raw text to determine the sentiment of the text, for example positive, negative, or neutral sentiment.
Document Classification is different than Sentiment analysis in that document classification does not necessarily have sentiment, as this assignment is a prime example whereas sentiment analysis is perceived to be a form of document classification, as it classifies sentiment.
To create labeled sentiment data, it is possible to use libraries, or create your own manually. The labels are not necessarily sentiment, but weights and scores to determine sentiment.
Common applications of sentiment analysis include social media websites monitoring user inputs to flag media that violate user terms for further analysis and removal. Sentiment analysis is also useful in algorithmic trading and marketing.

- 2) Research our why word embeddings worked better for the lecture notebook than on the whiskey competition.

text classification works better on some data than others. From the google text classification documentation, "From our experiments, we have observed that the ratio of “number of samples” (S) to “number of words per sample” (W) correlates with which model performs well."
Neural Networks are becoming more popular for document classification as computers become more powerful. Also, recurrent neural networks are useful in both classification and regression tasks, which is also useful in document classication and predictions